In [4]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%pprint
%matplotlib inline
import matplotlib.pyplot as plt
import re
from math import cos, sin, pi, sqrt, atan
import numpy as np
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 atan	 attempt_to_pickle	 cos	 
csv	 example_iterrows	 get_column_descriptions	 label_arc	 load_csv	 load_dataframes	 load_object	 logging	 math	 
np	 os	 pd	 pi	 pickle	 plt	 re	 save_dataframes	 sin	 
sqrt	 store_objects	 sys	 


In [2]:

ccw_history_radius_dict = load_object('ccw_history_radius_dict')
cw_history_radius_dict = load_object('cw_history_radius_dict')
ccw_history_year_dict = load_object('ccw_history_year_dict')
cw_history_year_dict = load_object('cw_history_year_dict')
ccw_history_theta_dict = load_object('ccw_history_theta_dict')
cw_history_theta_dict = load_object('cw_history_theta_dict')
saeculum_cmap_dict = load_object('saeculum_cmap_dict')
dataframes_dict = load_dataframes(generations_df='generations_df', patriline_df='patriline_df',
                                  turnings_df='turnings_df')

In [ ]:

index_name = 'Patriarch Name'
patriline_df = dataframes_dict['patriline_df']
if 'Unnamed: 0' in patriline_df.columns:
    patriline_df = patriline_df.set_index('Unnamed: 0')
elif index_name in patriline_df.columns:
    patriline_df = patriline_df.set_index(index_name)
patriline_df.index.name = 'patriarch_name'

In [ ]:

patriline_df.sample(5).T

In [ ]:

index_name = 'generation_name'
generations_df = dataframes_dict['generations_df']
if 'Unnamed: 0' in generations_df.columns:
    generations_df = generations_df.set_index('Unnamed: 0')
elif index_name in generations_df.columns:
    generations_df = generations_df.set_index(index_name)
generations_df.index.name = index_name

In [ ]:

generations_df.sample(5).T

In [ ]:

min_year = patriline_df['Year of Birth'].min()
match_series = (generations_df['birth_year_begin'] >= min_year)
store_objects(patriline_df=patriline_df, generations_df=generations_df, turnings_df=turnings_df)
generations_df[match_series]['saeculum_name'].unique()

In [ ]:

index_name = 'turning_name'
turnings_df = dataframes_dict['turnings_df']
if 'Unnamed: 0' in turnings_df.columns:
    turnings_df = turnings_df.set_index('Unnamed: 0')
elif index_name in turnings_df.columns:
    turnings_df = turnings_df.set_index(index_name)
turnings_df.index.name = index_name

In [ ]:

turnings_df.sample(5).T

In [ ]:

min_year = patriline_df['Year of Birth'].min()
match_series = (turnings_df['turning_begin_year'] >= min_year)
turnings_df[match_series]['saeculum_name'].unique()

In [ ]:

turnings_df['turning_end_year'].unique()

In [ ]:

columns_list = ['turning_begin_year', 'turning_end_year']
turnings_df[columns_list].apply(lambda row_series: int((row_series['turning_begin_year']+row_series['turning_end_year'])/2), axis=1)

In [ ]:

from pathlib import Path

png_folder = os.path.join(SAVES_FOLDER, 'png')
year = 1961
match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
saeculum_name = turnings_df[match_series].index.tolist()[0]
saeculum_file = '{}.jpg'.format(saeculum_name)
saeculum_path = Path(os.path.join(png_folder, saeculum_file))
dir(saeculum_path)

In [ ]:

os.path.abspath(saeculum_path)

In [ ]:

def polar_to_cartesian(r, theta):
    radians = theta*(pi/180)
    
    return int(r*cos(radians)), int(r*sin(radians))

In [ ]:

def create_xy_list(history_radius_dict):
    xy_list = []
    for radius in sorted(history_radius_dict.keys()):
        year, theta = history_radius_dict[radius]
        cartesian_tuple = polar_to_cartesian(radius, theta)
        if len(xy_list):
            if (cartesian_tuple != xy_list[-1]):
                xy_list.append(cartesian_tuple)
        else:
            xy_list.append(cartesian_tuple)
    
    return xy_list

In [ ]:

def get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i=0):
    xy_list = []
    i = i % 4
    if stop_year > stopped_year:
        stop_year = stopped_year
    start_radius = history_year_dict[start_year][0]
    start_radius += 25*i
    stop_radius = history_year_dict[stop_year][0]
    stop_radius += 25*i
    increment_count = int(2*pi*start_radius)
    radius_array = np.linspace(start=start_radius, stop=stop_radius,
                               num=increment_count)
    start_theta = history_year_dict[start_year][1]
    stop_theta = history_year_dict[stop_year][1]
    theta_array = np.linspace(start=start_theta, stop=stop_theta,
                              num=increment_count)
    for radius, theta in zip(radius_array, theta_array):
        cartesian_tuple = polar_to_cartesian(radius, theta)
        if len(xy_list):
            if (cartesian_tuple != xy_list[-1]):
                xy_list.append(cartesian_tuple)
        else:
            xy_list.append(cartesian_tuple)
    
    return xy_list

In [ ]:

from PIL import Image
from matplotlib.pyplot import imshow

jpg_dir = os.path.join(SAVES_FOLDER, 'jpg')
MAX_WIDTH = 936
MAX_HEIGHT = 936
def show_year_image(year):
    file_name = '{}.jpg'.format(year)
    file_path = os.path.join(jpg_dir, file_name)
    jpg_image = Image.open(fp=file_path, mode='r')
    jpg_image = jpg_image.rotate(angle=180)
    width, height = jpg_image.size
    if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
        if (width > MAX_WIDTH):
            multiple = MAX_WIDTH / width
            width *= multiple
            height *= multiple
        if (height > MAX_HEIGHT):
            multiple = MAX_HEIGHT / height
            width *= multiple
            height *= multiple
        jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [ ]:

def show_turning_image(year):
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    turning_name_list = turnings_df[match_series].index.tolist()
    if len(turning_name_list):
        turning_name = turning_name_list[0]
        file_name = '{}.jpg'.format(turning_name)
        file_path = os.path.join(jpg_dir, file_name)
        jpg_image = Image.open(fp=file_path, mode='r')
        jpg_image = jpg_image.rotate(angle=180)
        width, height = jpg_image.size
        if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
            if (width > MAX_WIDTH):
                multiple = MAX_WIDTH / width
                width *= multiple
                height *= multiple
            if (height > MAX_HEIGHT):
                multiple = MAX_HEIGHT / height
                width *= multiple
                height *= multiple
            jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [ ]:

def show_saeculum_image(saeculum_name):
    file_name = '{}.jpg'.format(saeculum_name)
    file_path = os.path.join(jpg_dir, file_name)
    jpg_image = Image.open(fp=file_path, mode='r')
    jpg_image = jpg_image.rotate(angle=180)
    width, height = jpg_image.size
    if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
        if (width > MAX_WIDTH):
            multiple = MAX_WIDTH / width
            width *= multiple
            height *= multiple
        if (height > MAX_HEIGHT):
            multiple = MAX_HEIGHT / height
            width *= multiple
            height *= multiple
        jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [ ]:

import re

dresses_list = []
dresses_regex = re.compile(r'(\d{4})-(\d{4})\.png')
first_date_dict = {}
second_date_dict = {}
for sub_dir, dirs_list, files_list in os.walk(png_dir):
    for file_name in files_list:
        if dresses_regex.fullmatch(file_name):
            match_obj = dresses_regex.search(file_name)
            if match_obj:
                first_date = int(match_obj.group(1))
                second_date = int(match_obj.group(2))
                diff = second_date - first_date
                print('{}: {}'.format(file_name, diff))
                if diff < 10:
                    dresses_list.append(file_name)
                    first_date_dict[first_date] = file_name
                    second_date_dict[second_date] = file_name

In [ ]:

store_objects(dresses_list=dresses_list, first_date_dict=first_date_dict, second_date_dict=second_date_dict)
dresses_list

In [ ]:

max(last_years_list)

In [ ]:

first_years_list = list(first_date_dict.keys())
last_years_list = list(second_date_dict.keys())
dresses_file_dict = {}
for stopped_year in range(min_year, 2031):
    if stopped_year < min(first_years_list):
        dresses_file = None
    elif (stopped_year in first_years_list):
        dresses_file = first_date_dict[stopped_year]
    elif (stopped_year in last_years_list):
        dresses_file = second_date_dict[stopped_year]
    elif stopped_year > max(last_years_list):
        dresses_file = None
    dresses_file_dict[stopped_year] = dresses_file
store_objects(dresses_file_dict=dresses_file_dict)

In [ ]:

white_tuple = (255, 255, 255, 0)
def add_fashion_image(year):
    
    # Get old image data
    old_path = os.path.join(png_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    old_data_list = foreground.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        if (old_tuple[0] == 255) and (old_tuple[1] == 255) and (old_tuple[2] == 255):
            new_data_list.append(white_tuple)
        else:
            new_data_list.append(old_tuple)
    
    # Replace old with new
    foreground.putdata(new_data_list)
    
    # Get dresses image
    dresses_file = dresses_file_dict[year]
    if dresses_file is None:
        file_path = os.path.join(png_folder, 'plot_{}_fashionable.png'.format(year))
        foreground.save(file_path, 'PNG')
    else:
        file_path = os.path.join(png_folder, dresses_file)
        dresses_img = Image.open(file_path)
        dresses_img = dresses_img.convert('RGBA')

        dresses_img.paste(foreground, (0, 0), foreground)

        file_path = os.path.join(png_folder, 'plot_{}_fashionable.png'.format(year))
        dresses_img.save(file_path, 'PNG')

In [ ]:

from pathlib import Path

saeculum_dir = os.path.join(DATA_FOLDER, 'saeculum')
movie_folder = os.path.join(SAVES_FOLDER, 'movies')
saeculum_movie_folder = os.path.join(movie_folder, 'saeculum')
bare_movie_folder = os.path.join(movie_folder, 'bare')
os.makedirs(name=saeculum_movie_folder, exist_ok=True)
MAX_WIDTH = 936
MAX_HEIGHT = 936
def add_saeculum_image(year):
    
    # Get old image data
    old_path = os.path.join(bare_movie_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    old_data_list = foreground.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        if (old_tuple[0] == 255) and (old_tuple[1] == 255) and (old_tuple[2] == 255):
            new_data_list.append(white_tuple)
        else:
            new_data_list.append(old_tuple)
    
    # Replace old with new
    foreground.putdata(new_data_list)
    
    # Get saeculum image
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    saeculum_list = turnings_df[match_series].index.tolist()
    if len(saeculum_list):
        saeculum_name = saeculum_list[0]
        saeculum_file = '{}.png'.format(saeculum_name)
        saeculum_path = Path(os.path.join(saeculum_dir, saeculum_file))
        new_path = os.path.join(saeculum_movie_folder, 'plot_{}_saeculum.png'.format(year))
        if saeculum_path.is_file():
            saeculum_img = Image.open(saeculum_path, mode='r')
            saeculum_img = saeculum_img.convert('RGBA')
            saeculum_img.paste(foreground, (0, 0), foreground)
            saeculum_img.save(new_path, 'PNG')
        else:
            foreground.save(new_path, 'PNG')

In [ ]:

dresses_list = load_object('dresses_list')
first_date_dict = load_object('first_date_dict')
second_date_dict = load_object('second_date_dict')
dresses_file_dict = load_object('dresses_file_dict')

In [ ]:

movie_folder = os.path.join(SAVES_FOLDER, 'movies')
bare_movie_folder = os.path.join(movie_folder, 'bare')

In [5]:

patriline_df = load_object('patriline_df')
def show_stopped_babbitt_plot(history_year_dict, history_theta_dict, stopped_year):

    # Turn interactive plotting off
    plt.ioff()
    
    # Create a new figure, plot into it, then close it so it never gets displayed
    fig = plt.figure(figsize=(13, 13))
    ax = fig.add_subplot(111, autoscale_on=False)
    ax.set_xlim(-1000, 1000)
    ax.set_ylim(-1000, 1000)
    match_series = (patriline_df['Year of Birth'] <= stopped_year)
    i = patriline_df[match_series].shape[0]-1
    d = 5
    previous_saeculum = patriline_df[match_series].head(1)['Saeculum Name'].tolist()[0]
    for patriarch_name, row_series in patriline_df[match_series].iterrows():
        start_year = int(row_series['Year of Birth'])
        stop_year = row_series['Year of Death']
        try:
            stop_year = int(stop_year)
        except:
            stop_year = start_year + 80
            if stop_year > max(history_year_dict.keys()):
                stop_year = max(history_year_dict.keys())
        xy_list = get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i)
        years_list = [year for year in [start_year, stop_year] if year <= stopped_year]
        add_spiral_labels(years_list, history_year_dict, i)
        if stop_year > stopped_year:
            stop_year = stopped_year
        text_color = row_series['xkcd_color']
        label_arc(start_year, stop_year, history_year_dict, history_theta_dict, patriarch_name, 13, i, text_color)
        saeculum = row_series['Saeculum Name']
        if saeculum != previous_saeculum:
            previous_saeculum = saeculum
            d = 5
        #print(patriarch_name, i, d, saeculum)
        cmap = saeculum_cmap_dict[saeculum]
        c = plt.get_cmap(cmap)(np.linspace(0, 1, 6))[d]
        PathCollection_obj = plt.plot([xy[0] for xy in xy_list], [xy[1] for xy in xy_list],
                                      alpha=0.75, label=patriarch_name, c=c, linewidth=9)
        i -= 1
        d -= 1
    
    #show_turning_image(stopped_year)
    legend_obj = ax.legend()
    frame_obj = legend_obj.get_frame()
    frame_obj.set_facecolor('whitesmoke')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Close it so it never gets displayed
    file_name = 'plot_{}.png'.format(stopped_year)
    file_path = os.path.join(bare_movie_folder, file_name)
    plt.savefig(file_path, format='png')
    plt.close(fig)

In [ ]:

def add_spiral_labels(years_list, history_year_dict, i=0):
    i = i % 4
    for year in years_list:
        radius, theta = history_year_dict[year]
        radius += 25*i
        radius -= 25/2
        x, y = polar_to_cartesian(radius, theta)
        text_obj = plt.text(x, y, year, fontsize=10, color='gray',
                            rotation=theta-90, rotation_mode='anchor')

In [1]:

import logging

logging.basicConfig(filename='label_arc.log', filemode='a', format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S', level=logging.DEBUG)
logging.info('Running Label Arc')
def label_arc(start_year, stopped_year, history_year_dict, history_theta_dict,
              arc_label, ideal_distance=13, i=0, label_color='black'):
    i = i % 4
    starting_year = int(((start_year + stopped_year) / 2) - (len(arc_label) / 2))
    starting_radius, starting_theta = history_year_dict[starting_year]
    next_radius, next_theta = history_year_dict[starting_year+1]
    

    # Tan(A) = Opposite/Adjacent
    radians = atan(ideal_distance/starting_radius)
    theta_sign = np.sign(next_theta-starting_theta)
    
    # Increment the theta so that it spaces the letters the same regardless of the radius
    theta_increment = theta_sign*(radians*180/pi)
    
    # Figure out if you have to flip the characters upside-down and place them backwards
    if (starting_theta%360) < 200:
        logging.info('')
        logging.info('Right-side up thetas:')
        theta = starting_theta
        radius = starting_radius + 25*i
        radius -= 25/2
        for c in arc_label[::int(-theta_sign)]:
            logging.info('c: "{}", radius: "{}", theta: "{}"'.format(c, radius, theta % 360))
            x, y = polar_to_cartesian(radius, theta)
            text_obj = plt.text(x, y, c, fontsize=12, color=label_color,
                                rotation=theta-90, rotation_mode='anchor')
            theta += theta_increment
            if int(theta) in history_theta_dict:
                radius = history_theta_dict[int(theta)][1]
                radius += 25*i
                radius -= 25/2
    else:
        logging.info('')
        logging.info('Upside-down thetas:')
        theta = starting_theta + theta_increment*len(arc_label)
        radius = starting_radius + 25*i
        radius += 25/2
        for c in arc_label[::int(-theta_sign)]:
            logging.info('c: "{}", radius: "{}", theta: "{}"'.format(c, radius, theta % 360))
            x, y = polar_to_cartesian(radius, theta)
            text_obj = plt.text(x, y, c, fontsize=12, color=label_color,
                                rotation=theta+90, rotation_mode='anchor')
            theta -= theta_increment
            if int(theta) in history_theta_dict:
                radius = history_theta_dict[int(theta)][1]
                radius += 25*i
                radius += 25/2

In [ ]:

show_stopped_babbitt_plot(cw_history_year_dict, cw_history_theta_dict, 2020)

In [ ]:

logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:

png_dir = os.path.join(SAVES_FOLDER, 'png')
os.makedirs(name=png_dir, exist_ok=True)
min_year = patriline_df['Year of Birth'].min()
for stopped_year in range(min_year, 2031):
    show_stopped_babbitt_plot(cw_history_year_dict, cw_history_theta_dict, stopped_year)

In [ ]:

def distance_from_white(old_tuple):
    x_diff = 255 - old_tuple[0]
    y_diff = 255 - old_tuple[1]
    z_diff = 255 - old_tuple[2]
    
    return sqrt(x_diff**2 + y_diff**2 + z_diff**2)

In [ ]:

def distance_from_black(old_tuple):
    
    return sqrt(old_tuple[0]**2 + old_tuple[1]**2 + old_tuple[2]**2)

In [ ]:

white_tuple = (255, 255, 255, 0)
black_tuple = (0, 0, 0, 255)
margin = 255/distance_from_white(black_tuple)
def make_transparent(img, threshold=38):
    old_data_list = img.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        transparency = int(margin * distance_from_white(old_tuple))
        if transparency > threshold:
            transparency = 255
        elif transparency < 0:
            transparency = 0
        old_tuple = (old_tuple[0], old_tuple[1], old_tuple[2], transparency)
        new_data_list.append(old_tuple)
    
    # Replace old with new
    img.putdata(new_data_list)
    
    return img

In [ ]:

for i in range(1971, 1980):
    dresses_file_dict[i] = '1971-1979.png'

In [ ]:

from pathlib import Path

saeculum_fashionable_movie_folder = os.path.join(movie_folder, 'saeculum_fashionable')
os.makedirs(name=saeculum_fashionable_movie_folder, exist_ok=True)
def add_fashion_saeculum_image(year):
    
    # Get old image data
    old_path = os.path.join(bare_movie_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    foreground = make_transparent(foreground, threshold=6)
    
    # Get dresses image
    dresses_file = dresses_file_dict[year]
    if dresses_file is None:
        dresses_img = foreground
    else:
        file_path = os.path.join(png_folder, dresses_file)
        dresses_img = Image.open(file_path)
        dresses_img = dresses_img.convert('RGBA')
        dresses_img = make_transparent(dresses_img, threshold=5)
        dresses_img.paste(foreground, (0, 0), foreground)
    
    # Get saeculum image
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    saeculum_list = turnings_df[match_series].index.tolist()
    if len(saeculum_list):
        saeculum_name = saeculum_list[0]
        saeculum_file = '{}.png'.format(saeculum_name)
        saeculum_path = Path(os.path.join(saeculum_dir, saeculum_file))
        if saeculum_path.is_file():
            saeculum_img = Image.open(saeculum_path, mode='r')
            saeculum_img = saeculum_img.convert('RGBA')
            saeculum_img.paste(dresses_img, (0, 0), dresses_img)
        else:
            saeculum_img = dresses_img
        file_path = os.path.join(saeculum_fashionable_movie_folder, 'plot_{}_saeculum_fashionable.png'.format(year))
        saeculum_img.save(file_path, 'PNG')

In [ ]:

add_fashion_saeculum_image(1971)

In [ ]:

for stopped_year in range(1972, 2031):
    add_fashion_saeculum_image(stopped_year)

In [ ]:

min_year = patriline_df['Year of Birth'].min()
for stopped_year in range(min_year, 2031):
    add_fashion_saeculum_image(stopped_year)

In [ ]:

def make_a_movie(stop_year=2020, png_suffix='', movie_folder=bare_movie_folder, movie_name=None):
    images_list = []
    for stopped_year in range(min_year, stop_year):
        png_name = 'plot_{}{}.png'.format(stopped_year, png_suffix)
        png_path = os.path.join(movie_folder, png_name)
        images_list.append(imageio.imread(png_path))
    png_name = 'plot_{}{}.png'.format(stop_year, png_suffix)
    png_path = os.path.join(movie_folder, png_name)
    for i in range(200):
        images_list.append(imageio.imread(png_path))
    if movie_name is None:
        movie_name = '_'.join(png_suffix.split('_')[1:] + ['movie']) + '.gif'
    gif_path = os.path.join(movie_folder, movie_name)
    imageio.mimsave(gif_path, images_list)

In [ ]:

make_a_movie(2020, png_suffix='_saeculum_fashionable', movie_folder=saeculum_fashionable_movie_folder, movie_name='movie_to_2020.gif')

In [ ]:

make_a_movie(1989, png_suffix='_saeculum_fashionable', movie_folder=saeculum_fashionable_movie_folder, movie_name='stephen_movie.gif')

In [ ]:

import requests

def exists(path):
    r = requests.head(path)
    
    return r.status_code == requests.codes.ok

min_decade = 10 * (min_year//10)
for i in range(min_decade, 2020, 10):
    url = 'https://en.wikipedia.org/wiki/{}s_in_fashion'.format(i)
    if exists(url):
        print('{} exists'.format(i))

In [ ]:

plt.savefig?

In [ ]:

make_a_movie(1989)

In [ ]:

images_list = []
for stopped_year in range(min_year, 2020):
    png_name = 'plot_{}_saeculum_fashionable.png'.format(stopped_year)
    png_path = os.path.join(saeculum_fashionable_movie_folder, png_name)
    images_list.append(imageio.imread(png_path))
png_name = 'plot_{}_saeculum_fashionable.png'.format(2020)
png_path = os.path.join(saeculum_fashionable_movie_folder, png_name)
for i in range(100):
    images_list.append(imageio.imread(png_path))
gif_path = os.path.join(saeculum_fashionable_movie_folder, 'saeculum_fashionable_movie.gif')
imageio.mimsave(gif_path, images_list)
# Too busy to display in this cell
#display.Image(gif_path)

In [ ]:

import os
import imageio
from IPython import display

gif_dir = os.path.join(SAVES_FOLDER, 'gif')
os.makedirs(name=gif_dir, exist_ok=True)
images_list = []
for stopped_year in range(min_year, 2020):
    png_name = 'plot_{}_saeculum.png'.format(stopped_year)
    png_path = os.path.join(saeculum_movie_folder, png_name)
    images_list.append(imageio.imread(png_path))
png_name = 'plot_{}_saeculum.png'.format(2020)
png_path = os.path.join(saeculum_movie_folder, png_name)
for i in range(100):
    images_list.append(imageio.imread(png_path))
gif_path = os.path.join(saeculum_movie_folder, 'saeculum_movie.gif')
imageio.mimsave(gif_path, images_list)
# Too busy to display in this cell
#display.Image(gif_path)

In [ ]:

print(os.path.abspath(os.path.join(saeculum_movie_folder, 'plot_{}_saeculum.png'.format(2020))))

In [ ]:

import imageio
from IPython import display

gif_dir = os.path.join(SAVES_FOLDER, 'gif')
os.makedirs(name=gif_dir, exist_ok=True)
images = []
for stopped_year in range(min_year, 2020):
    file_name = 'plot_{}_fashionable.png'.format(stopped_year)
    file_path = os.path.join(SAVES_FOLDER, 'png', file_name)
    images.append(imageio.imread(file_path))
file_name = 'plot_{}_fashionable.png'.format(2020)
file_path = os.path.join(SAVES_FOLDER, 'png', file_name)
for i in range(100):
    images.append(imageio.imread(file_path))
file_path = os.path.join(gif_dir, 'fashionable_movie.gif')
imageio.mimsave(file_path, images)
display.Image(file_path)

In [ ]:

import sys

# Pillow
!{sys.executable} -m pip install --upgrade Pillow

In [ ]:

print('{} -m pip install --upgrade Pillow'.format(sys.executable))

In [ ]:

from PIL import Image, ImageFont, ImageDraw

png_folder = os.path.join(SAVES_FOLDER, 'png')
file_path = os.path.join(png_folder, 'plot_2019.png')
img_2019 = Image.open(file_path)
print(type(img_2019))
print([fn for fn in dir(img_2019) if 'get' in fn.lower()])

In [ ]:

img_2019.getbbox()

In [ ]:

exif_obj = img_1784_1866.getexif()
print(dir(exif_obj))
for key, value in exif_obj.items():
    print(key, value)

In [ ]:

items_view = exif_obj.items()
dir(items_view)

In [ ]:

bbox_tuple = img_1784_1866.getbbox()

In [ ]:

file_path = r'C:\Users\dev\Downloads\1784-1866.png'
img_1784_1866 = Image.open(file_path)
bbox_left, bbox_top, bbox_right, bbox_bottom = img_1784_1866.getbbox()
slice_count = int(bbox_bottom / 12)
for i in range(0, bbox_bottom, slice_count):
    crop_tuple = (bbox_left, i, bbox_right, i+slice_count)
    img_1784_1866.crop(crop_tuple).save(fp='{}_{}.png'.format(file_path.split('.')[0], i))

In [ ]:

'{}_{}.png'.format(file_path.split('.')[0], i)

In [ ]:

img_2019.getextrema()

In [ ]:

exif_obj = img_2019.getexif()
print(dir(exif_obj))
for key, value in exif_obj.items():
    print(key, value)

In [ ]:

img_2019.getdata(band=img_2019.getbands().index('A'))

In [ ]:

img_2019.getbands()

In [ ]:

img_2019.get_format_mimetype()

In [ ]:

w,h= img.size

# put pixels into 2D array for ease of use
data = list(img.getdata())
xy_data = []
for y in xrange(h):
    temp = []
    for x in xrange(w):
        temp.append(data[y*w + x])
    xy_data.append(temp)

# get the title
title = raw_input("Title:")

# load the font
font_size = 20
font = ImageFont.truetype("/path/to/font.ttf",font_size)

#  Get the required height for you images
height_needed = font.getsize(title)[1] + 2  # 2 px for padding

# get the upperleft pixel to match color
bg = xy_data[0][0]

# add rows to the data to prepare for the text
xy_data = [[bg]*w for i in range(height_needed+5)] + xy_data  # +5 for more padding

# resize image
img = img.resize((w,h+height_needed+5))

# convert data back to 1D array
data = []
for line in xy_data:
    data += line

# put the image back in the data
img.putdata(data)

# get the ImageDraw item for this image
draw = ImageDraw.Draw(img)

# draw the text
draw.text((5,0),title,font=font,fill=(0,0,0))  # fill is black

img.save('titled_plot.png')

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body;
var attribs = body.attributes;
var command = "notebook_path = " + "'"+attribs['data-notebook-path'].value+"'";
kernel.execute(command);

In [ ]:
# %load ../../load_magic/nbviewer.py

from IPython.display import HTML

notebook_viewer_url = 'https://nbviewer.jupyter.org/github/dbabbitt/notebooks/blob/master/'
notebook_viewer_url += '/'.join(notebook_path.split('/')[1:])
html_str = 'Click <a href="{}" target="_blank">here</a> to view notebook in nbviewer.'
HTML(html_str.format(notebook_viewer_url))